In [1]:
from pathlib import Path

from arcgis.features import GeoAccessor
from dm import Country
from dm.country import DemographicModeling

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

In [2]:
project_parent = Path('./').absolute().parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

biz_drop_cols = ['OBJECTID', 'CONAME','SALESVOL', 'HDBRCH', 'ULTNUM', 'PUBPRV', 'EMPNUM', 'FRNCOD', 'ISCODE', 'SQFTCODE', 'LOC_NAME', 'STATUS', 'SCORE', 'SOURCE', 'REC_TYPE']

In [3]:
usa = Country('USA')

usa

<dm.Country - USA (local)>

In [4]:
%%time
aoi_df = usa.cbsas.get('seattle')

aoi_df

Wall time: 5.16 s


,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-122.62951999978839, 47.163890001..."


In [5]:
%%time
biz_df = usa.business.get_by_name('ace hardware', aoi_df).drop(columns=biz_drop_cols)
biz_df.spatial.set_geometry('SHAPE')

biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               33 non-null     object  
 1   STREET               33 non-null     object  
 2   CITY                 33 non-null     object  
 3   STATE                33 non-null     object  
 4   STATE_NAME           33 non-null     object  
 5   ZIP                  33 non-null     object  
 6   ZIP4                 33 non-null     object  
 7   NAICS                33 non-null     object  
 8   SIC                  33 non-null     object  
 9   SHAPE                33 non-null     geometry
 10  id                   33 non-null     object  
 11  brand_name           33 non-null     object  
 12  brand_name_category  33 non-null     object  
dtypes: geometry(1), object(12)
memory usage: 3.5+ KB
Wall time: 3.36 s


In [6]:
%%time
comp_df = usa.business.get_competition(biz_df, aoi_df, local_threshold=3).drop(columns=biz_drop_cols)
comp_df.spatial.set_geometry('SHAPE')

comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               76 non-null     object  
 1   STREET               76 non-null     object  
 2   CITY                 76 non-null     object  
 3   STATE                76 non-null     object  
 4   STATE_NAME           76 non-null     object  
 5   ZIP                  76 non-null     object  
 6   ZIP4                 76 non-null     object  
 7   NAICS                76 non-null     object  
 8   SIC                  76 non-null     object  
 9   SHAPE                76 non-null     geometry
 10  id                   76 non-null     object  
 11  brand_name           76 non-null     object  
 12  brand_name_category  76 non-null     object  
dtypes: geometry(1), object(12)
memory usage: 7.8+ KB
Wall time: 1.6 s


In [ ]:
%%time
near_df = biz_df.dm.get_nearest(comp_df, source=usa, destination_count=6, near_prefix='near', destination_columns_to_keep=['NAICS', 'brand_name', 'brand_name_category'], single_row_per_origin=False)

In [15]:
near_df.head()

,LOCNUM,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,SHAPE,...,near_brand_name_05,near_brand_name_category_05,near_destination_id_06,near_proximity_kilometers_06,near_proximity_minutes_06,near_proximity_side_street_left_06,near_proximity_side_street_right_06,near_NAICS_06,near_brand_name_06,near_brand_name_category_06
0,216082099,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,525104,"{""x"": -122.43443850000003, ""y"": 47.08713600000...",...,CASCADE WHOLESALE HARDWARE,local_brand,175096239,15.161768,18.717571,0,1,44413005,LAKEWOOD HARDWARE & PAINT,local_brand
1,371889957,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,525104,"{""x"": -122.29924949999995, ""y"": 47.05465500000...",...,CENTRICSIT,local_brand,002890986,21.095097,26.426051,1,0,44413005,MC LENDON HARDWARE,local_brand
2,460556608,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,525104,"{""x"": -122.54611950000002, ""y"": 47.18077199999...",...,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC,715857966,16.018472,18.036884,0,1,44413005,WILCO FARM STORE,local_brand
3,405129289,POINT FOSDICK DR,GIG HARBOR,WA,Washington,98335,1711,44413005,525104,"{""x"": -122.58131400000003, ""y"": 47.30224499999...",...,SGS HARDWARE,local_brand,421117196,18.254821,16.836483,0,1,44413005,SGS HARDWARE,local_brand
4,404324160,S 12TH ST,TACOMA,WA,Washington,98405,3004,44413005,525104,"{""x"": -122.463747, ""y"": 47.24991900000006, ""sp...",...,SGS HARDWARE,local_brand,700565109,6.042069,10.175066,0,1,44413005,S G S HARDWARE,local_brand


In [18]:
near_cols = [c for c in near_df.columns if c.startswith('near_')]
near_df.iloc[0][['id'] + near_cols]

id                                                       216082099
near_destination_id_01                                   804462166
near_proximity_kilometers_01                               5.24511
near_proximity_minutes_01                                  6.90392
near_proximity_side_street_left_01                               1
near_proximity_side_street_right_01                              0
near_NAICS_01                                             44413005
near_brand_name_01                     BROOKDALE LUMBER TRUE VALUE
near_brand_name_category_01                            local_brand
near_destination_id_02                                   401706143
near_proximity_kilometers_02                                13.365
near_proximity_minutes_02                                  15.5373
near_proximity_side_street_left_02                               1
near_proximity_side_street_right_02                              0
near_NAICS_02                                             4441